In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import math
import h5py
from keras.utils.io_utils import HDF5Matrix
from datetime import datetime, timedelta
from PIL import Image
from pathlib import Path

from math import sqrt
import os.path
import csv
from csv import writer

Using TensorFlow backend.


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [2]:
def unique_value(array, property):
    array = array[property]
    array = array.drop_duplicates(keep='last')
    array = array.tolist()
    array.sort()
    return array

In [3]:
max_value = 0
max_path = 0

In [4]:
# setting general
name = '../../covid_dataset/9th_epoch200_relu/' 
size = 256
kernel_size = 64
weight = 5

# setting path
patient_daily_path = 'patient_daily/'
patient_daily_3days_accumulated_path = 'patient_daily_3days_accumulated/'
all_daily_path = 'all_daily/'
figure_path = 'figure/'
dataset_path = 'merged.h5'
dataset_figure_path = 'figure.h5'
model_path = 'model.h5'
pred_path = 'pred/'
x_test_path = 'x_test/'
y_test_path = 'y_test/'
y_pred_path = 'pred/'
readme_path = 'README.md'
scaled_path = 'scaled/'
diff_path = 'diff/'

# getting raw dataset
routes = pd.read_csv('../../covid_dataset/raw/MergedRoute.csv')
dates = unique_value(routes, 'date')
patients = unique_value(routes, 'patient_id')

# getting figure dataset
figure_routes = pd.read_csv('../../covid_dataset/raw/Figure.csv')
figure_dates = unique_value(figure_routes, 'date')
figure_patients = unique_value(figure_routes, 'patient_id')

# dataset info
first_day = datetime.strptime(dates[0], "%Y-%m-%d")
last_day = datetime.strptime(dates[-1], "%Y-%m-%d") # + timedelta(days=3)
delta = last_day - first_day
duration = delta.days + 1

# setting features
names = ['age', 'sex', 'infection_case', 'type', 'date']
counts = [11, 2, 4, 21, 7]
visit_types = ['karaoke', 'gas_station', 'gym', 'bakery', 'pc_cafe',
              'beauty_salon', 'school', 'church', 'bank', 'cafe',
              'bar', 'post_office', 'real_estate_agency', 'lodging',
              'public_transportation', 'restaurant', 'etc', 'store',
              'hospital', 'pharmacy', 'airport']
causes = ['community infection', 'etc', 'contact with patient', 'overseas inflow']

# setting model
channel = sum(count for count in counts)
split_num = 0
n_step = 3
epochs = 200
batch_size = 1
optimizer = 'rmsprop' # 'adadelta'
loss = 'mean_squared_error' # 'binary_crossentropy'
activation = 'relu' # 'relu', 'sigmoid'

In [5]:
class Patient:
    def age_category(age):
        age = int(age[:-1])
        if age == 0: return 0
        elif age == 100: return 10
        return age // 10
            
    def sex_category(sex):
        if sex == 'male': return 0
        return 1

    def infection_case_category(infection_case, causes):
        return causes.index(infection_case)
    
    def type_category(visit_type, move_types):
        return move_types.index(visit_type)

    def day_category(day):
        day = datetime.strptime(day, "%Y-%m-%d")
        return day.weekday()

In [6]:
def combine_places(places, counts, causes, visit_types):
    indices = []
    for i in range(len(places)):
        one_visit = places.iloc[i]
        indices.append(df_to_grid_index(one_visit, counts, causes, visit_types))
    return indices

def df_to_grid_index(one_visit, counts, causes, visit_types):
    index = 0
    p_age = Patient.age_category(one_visit['age'])
    index += counts[0]
    p_sex = Patient.sex_category(one_visit['sex']) + index
    index += counts[1]
    p_infection_case = Patient.infection_case_category(one_visit['infection_case'], causes) + index
    index += counts[2]
    p_type = Patient.type_category(one_visit['type'], visit_types) + index
    index += counts[3]
    p_date = Patient.day_category(one_visit['date']) + index
    row = one_visit['row']
    col = one_visit['col']
    
    return [p_age, p_sex, p_infection_case, p_type, p_date, row, col]

def put_triangular_kernel(array, row, col, value):
    stride = int((kernel_size - 1) / 2)
    ratio = 1 / (stride + 1)
   
    for i in range(row - stride, row + stride + 1):
        if i < 0 or i >= array.shape[0]: continue
        for j in range(col - stride, col + stride + 1):
            if j < 0 or j >= array.shape[1]: continue
            distance = math.sqrt((row - i)**2 + (col - j)**2)
            new_value = value * (1 - (distance * ratio))
            if new_value < 0: new_value = 0
            array[i][j] = new_value
       
    array[row][col] = value
    return array

def overlay_kernel(array):
    new_image = np.zeros((array.shape[0],array.shape[1]))
    for row in range(array.shape[0]):
        for col in range(array.shape[1]):
            if array[row][col] == 0: continue
            new_image += put_triangular_kernel(np.zeros((array.shape[0], array.shape[1])), row, col, array[row][col])

    return new_image

def indices_save_image(path, place_indices):
    all_counts = sum(count for count in counts)
    visit_grid = np.zeros((all_counts, size, size))
    
    for index in place_indices:
        row = index[5]
        col = index[6]
        for feature in range(5):
            visit_grid[index[feature]][row][col] += weight
    
    for channel in range(visit_grid.shape[0]):
        save_grid(path + str(channel) + ".png", visit_grid[channel])
    
def save_grid(path, grid, kernel=True):
    global max_value, max_path
    if kernel: grid = overlay_kernel(grid)
    if np.amax(grid) > max_value:
        max_value = np.amax(grid)
        max_path = path
    img = Image.fromarray(grid.astype('uint8'), 'L')
    img.save(path)

### Raw Data 저장

In [7]:
def save_patient_route(path, patient, routes):
    patient_places = routes[routes['patient_id']==patient]
    patient_dates = unique_value(patient_places, 'date')
    f_first_day = datetime.strptime(patient_dates[0], "%Y-%m-%d")
    f_last_day = datetime.strptime(patient_dates[-1], "%Y-%m-%d") + timedelta(days=3)
    f_delta = f_last_day - f_first_day
    f_duration = f_delta.days + 1

    # 저장
    patient_path = path + str(patient)
    Path(patient_path).mkdir(parents=True, exist_ok=True)

    today = f_first_day
    while(True):
        today_str = datetime.strftime(today, "%Y-%m-%d")
        patient_day_places = patient_places[patient_places['date']==today_str]
        places_indices = combine_places(patient_day_places, counts, causes, visit_types)
        patient_date_path = patient_path + "/" + today_str + '/'
        Path(patient_date_path).mkdir(parents=True, exist_ok=True)
        indices_save_image(patient_date_path, places_indices)
        if today == f_last_day: break
        today += timedelta(days=1)

def get_patient_route(patient, routes):
    patient_places = routes[routes['patient_id']==patient]
    patient_dates = unique_value(patient_places, 'date')
    f_first_day = datetime.strptime(patient_dates[0], "%Y-%m-%d")
    f_last_day = datetime.strptime(patient_dates[-1], "%Y-%m-%d") + timedelta(days=3)
    f_delta = f_last_day - f_first_day
    f_duration = f_delta.days + 1

    patient_routes = []
    today = f_first_day
    while(True):
        today_str = datetime.strftime(today, "%Y-%m-%d")
        patient_day_places = patient_places[patient_places['date']==today_str]
        places_indices = combine_places(patient_day_places, counts, causes, visit_types)
        patient_routes.append([today_str, places_indices]) 

        if today == f_last_day: break
        today += timedelta(days=1)
        
    return patient_routes

In [8]:
#### raw data 저장

#### 확진자 기준으로 raw data 저장

# path = name + patient_daily_path
# for patient in patients:
#     save_patient_route(path, patient, routes)


#### figure 저장

# path = name + figure_path + patient_daily_path
# for patient in figure_patients:
#     save_patient_route(path, patient, figure_routes)


### 3일 누적 데이터 저장

In [9]:
def accumulate_two_days(day1, day2):
    day2[1].extend(day1[1])
    return day2
    
def accumulate_patient(patient, routes):
    patient_route = get_patient_route(patient, routes)
    patient_days = len(patient_route)

    second = patient_days - 1
    first = second - 1
    for i in range(2 * patient_days - 3):
        patient_route[second] = accumulate_two_days(patient_route[first], patient_route[second])
        if second - first == 2: second -=1
        else: first -= 1
    
    return patient_route

def save_patient_routes(path, patient, patient_routes):
    path += str(patient) + '/'
    Path(path).mkdir(parents=True, exist_ok=True)
    for routes in patient_routes:
        patient_date_path = path + routes[0] + '/'
        Path(patient_date_path).mkdir(parents=True, exist_ok=True)
        indices_save_image(patient_date_path, routes[1])

In [10]:
#### 누적 경로 저장

#### 확진자 기준으로 저장
# ```
# path = name + patient_daily_3days_accumulated_path
# for patient in patients:
#     accumulated_routes = accumulate_patient(patient, routes)
#     save_patient_routes(path, patient, accumulated_routes)
# ```

#### figure 저장    
# ```
# path = name + figure_path + patient_daily_3days_accumulated_path
# for patient in figure_patients:
#     accumulated_routes = accumulate_patient(patient, figure_routes)
#     save_patient_routes(path, patient, accumulated_routes)
# ```

### 각 날짜별 취합

In [11]:
def get_complete_routes(routes, dates, patients):
    f_first_day = datetime.strptime(dates[0], "%Y-%m-%d")
    f_last_day = datetime.strptime(dates[-1], "%Y-%m-%d") + timedelta(days=3)

    # 날짜별 경로 배열 생성
    today = f_first_day
    complete_routes = []
    while(True):
        today_str = datetime.strftime(today, "%Y-%m-%d")
        places = []
        complete_routes.append([today_str, places])

        if today == f_last_day: break
        today += timedelta(days=1)

    # 환자 경로 가져온 다음 날짜대로 배치
    for patient in patients:
        accumulated_routes = accumulate_patient(patient, routes)
        for each_route in accumulated_routes:
            route_day = datetime.strptime(each_route[0], "%Y-%m-%d")
            route_places = each_route[1]
            index = (route_day - f_first_day).days
            complete_routes[index][1].extend(route_places)
    return complete_routes

In [12]:
#### 취합한 경로 저장

#### 확진자 기준으로 저장
# ```
# path = name + all_daily_path
# complete_routes = get_complete_routes(routes, dates, patients)
# for days in complete_routes:
#     date_path = path + days[0] + '/'
#     Path(date_path).mkdir(parents=True, exist_ok=True)
#     indices_save_image(date_path, days[1])
# ```

#### figure 저장
# ```
# path = name + figure_path + all_daily_path
# complete_routes = get_complete_routes(figure_routes, figure_dates, figure_patients)
# for days in complete_routes:
#     date_path = path + days[0] + '/'
#     Path(date_path).mkdir(parents=True, exist_ok=True)
#     indices_save_image(date_path, days[1])
# ```

### 데이터셋 파일로 저장

In [13]:
def get_dataset(routes, dates, patients):
    complete_routes = get_complete_routes(routes, dates, patients)
    all_counts = sum(count for count in counts)
    dataset = np.zeros((len(complete_routes), all_counts, size, size))

    for i, days in enumerate(complete_routes):
        sub_routes = get_array_image(days, dataset[i,:,:,:])
    
    return dataset

def get_array_image(place_indices, data_array):
    all_counts = sum(count for count in counts)
    
    for index in place_indices[1]:
        row = index[5]
        col = index[6]
        for feature in range(5):
            data_array[index[feature]][row][col] += weight
    
    for channel in range(data_array.shape[0]):
        data_array[channel] = overlay_kernel(data_array[channel])
    
    return data_array
    
def divide_dataset(dataset):
    n = dataset.shape[0]
    channel = dataset.shape[1]

    X_set = []
    y_set = []
    for i in range(n_step, n):
        X_set.append(dataset[i-n_step:i, :,:])
        y_set.append(dataset[i:i+1, :,:])
    X_set, y_set = np.array(X_set), np.array(y_set)
   
    return X_set, y_set
    
def save_dataset(path, X_set, y_set, start_day):
    start_day = start_day.split('-')
    start_day = list(map(int, start_day))
    
    with h5py.File(path,'w') as f:    
        set_X = f.create_dataset('X_set', data=X_set)
        set_y = f.create_dataset('y_set', data=y_set)
        set_day = f.create_dataset('start_day', data=np.array(start_day))

In [14]:
#### save dataset

#### 확진자 기준으로 저장
# ```
# path = name + dataset_path
# dataset = get_dataset(routes, dates, patients)
# X_set, y_set = divide_dataset(dataset)
# save_dataset(path, X_set, y_set, dates[0])
# print(X_set.shape, y_set.shape)
# ```

#### figure 저장
# ```
# path = name + dataset_figure_path
# dataset = get_dataset(figure_routes, figure_dates, figure_patients)
# X_set, y_set = divide_dataset(dataset)
# save_dataset(path, X_set, y_set, figure_dates[0])
# print(X_set.shape, y_set.shape)
# ```

### ConvLSTM

In [15]:
from keras.models import Sequential, load_model
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
import keras.backend.tensorflow_backend as K
import tensorflow as tf
import keras.backend.tensorflow_backend as tfback

In [16]:
# We create a layer which take as input movies of shape
# (n_frames, width, height, channels) and returns a movie
# of identical shape.
def get_model(channel):
    with K.tf_ops.device('/GPU:0'):
        seq = Sequential()
        seq.add(ConvLSTM2D(filters=channel, kernel_size=(3, 3), data_format='channels_first',
                           input_shape=(n_step, channel, size, size),
                           padding='same', return_sequences=True))
        seq.add(BatchNormalization())

        seq.add(ConvLSTM2D(filters=channel, kernel_size=(3, 3), data_format='channels_first',
                           padding='same', return_sequences=True))
        seq.add(BatchNormalization())

        seq.add(ConvLSTM2D(filters=channel, kernel_size=(3, 3), data_format='channels_first',
                           padding='same', return_sequences=True))
        seq.add(BatchNormalization())

        seq.add(ConvLSTM2D(filters=channel, kernel_size=(3, 3), data_format='channels_first',
                           padding='same', return_sequences=True))
        seq.add(BatchNormalization())

        seq.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
                       activation=activation,
                       padding='same', data_format='channels_first'))
        
        seq.compile(optimizer=optimizer, loss=loss)
                
        seq.summary()

    return seq

#### train

In [17]:
def load_data(path, n_train):
    X_train = HDF5Matrix(path, 'X_set', start=0, end=n_train)
    y_train = HDF5Matrix(path, 'y_set', start=0, end=n_train)
    X_test = HDF5Matrix(path, 'X_set', start=n_train)
    y_test = HDF5Matrix(path, 'y_set', start=n_train)
    
    start_day = np.array(HDF5Matrix(path, 'start_day'))
    start_day = "%d-%.2d-%.2d" % (start_day[0], start_day[1], start_day[2])
    start_day1 = datetime.strptime(start_day, "%Y-%m-%d")
    start_day2 = start_day1 + timedelta(days=n_train)
    
    return X_train, y_train, X_test, y_test, start_day1, start_day2

def figure_load_data(path):
    X_test = HDF5Matrix(path, 'X_set')
    y_test = HDF5Matrix(path, 'y_set')
    start_day = np.array(HDF5Matrix(path, 'start_day'))
    start_day = "%d-%.2d-%.2d" % (start_day[0], start_day[1], start_day[2])
    start_day = datetime.strptime(start_day, "%Y-%m-%d")
    
    return X_test, y_test, start_day

def get_array_image(place_indices, data_array):
    all_counts = sum(count for count in counts)
    
    for index in place_indices[1]:
        row = index[5]
        col = index[6]
        for feature in range(5):
            data_array[index[feature]][row][col] += weight
    
    for channel in range(data_array.shape[0]):
        data_array[channel] = overlay_kernel(data_array[channel])
    
    return data_array

def train(path, X_train, y_train):
    seq = get_model(channel)
    seq.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, shuffle='batch')
    seq.save(path)

In [18]:
#### 데이터셋 불러오기

#### 확진자

path = name + dataset_path
sample_num = duration - (2 * n_step)
if split_num == 0: split_num = int(sample_num * 0.7)
X_train, y_train, X_test, y_test, start_day1, start_day2 = load_data(path, split_num)

#### figure

# path = name + dataset_figure_path
# X_figure, y_figure, start_figure = figure_load_data(path)


/home/vislab/.local/lib/python3.7/site-packages/keras/utils/io_utils.py:60: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File(datapath)


#### 트레이닝 함수 실행

path = name + model_path
train(path, X_train, y_train)


#### test

In [19]:
#### predict 함수 실행

#### 확진자

path = name + model_path
model = load_model(path)
pred = model.predict(X_test)
print(pred.shape)

#### figure

# pred_figure = model.predict(X_figure)
# print(pred_figure.shape)


(34, 1, 45, 256, 256)


In [79]:
def create_diff_file(file_path, file_name):
    Path(file_path).mkdir(parents=True, exist_ok=True)
    file_exists = os.path.isfile(file_path + file_name)
    with open (file_path + file_name, 'w') as csvfile:
        headers = ['date', 'feature', 'mape', 'rmse', 'mase', 'max']
        writer = csv.DictWriter(csvfile, delimiter=',', lineterminator='\n',fieldnames=headers)
        writer.writeheader()
            
def append_list_as_row(file_name, list_of_elem):
    print(list_of_elem[4])
    with open(file_name, 'a+', newline='') as write_obj:
        csv_writer = writer(write_obj)
        csv_writer.writerow(list_of_elem)

In [80]:
#### diff 구하기 - 확진자
#### MAPE 구하기 - 확진자
#### RMSE 구하기 - 확진자

diff = np.zeros((pred.shape[0], pred.shape[2], pred.shape[3], pred.shape[4]))
rmse = np.zeros((pred.shape[0], pred.shape[2]))
mape = np.zeros((pred.shape[0], pred.shape[2]))
mase = np.zeros((pred.shape[0], pred.shape[2]))
test_max_value = np.zeros((pred.shape[0], pred.shape[2]))
pred_max_value = np.zeros((pred.shape[0], pred.shape[2]))

for i in range(pred.shape[0]): # days
    for j in range(pred.shape[2]): # features
        pred_diff = pred[i][0][j]
        y_test_diff = y_test[i][0][j]
        diff[i,j,:,:] = 255 - np.abs(y_test_diff - pred_diff)
        
        pred_diff[pred_diff >= 0] += 1
        y_test_diff[y_test_diff >= 0] += 1
        
        rmse[i][j] = sqrt(mean_squared_error(y_test_diff, pred_diff))
        test_max_value[i][j] = np.amax(y_test[i][0][j])+1
        pred_max_value[i][j] = np.amax(pred[i][0][j])+1
        mape[i][j] = np.mean(np.abs((y_test_diff - pred_diff) / y_test_diff))
        if i == 0 and j == 1: print(np.amax(y_test[i][0][j]))


11.29039297460175


In [81]:
def save_prediction_image(path, X_test, y_test, pred, first_day):
    diff_save_path = path + diff_path
    diff_save_name = 'accuracy.csv'
    create_diff_file(diff_save_path, diff_save_name)
    
    # X_test
    l_first_day = first_day
    for l_sample in range(X_test.shape[0]):
        sample_path = path + x_test_path + 'sample%d/' % l_sample
        l_first_day2 = l_first_day
        for l_day in range(X_test.shape[1]):
            day_path = sample_path + datetime.strftime(l_first_day2, "%Y-%m-%d") + '/'
            Path(day_path).mkdir(parents=True, exist_ok=True)
            array_save_image(day_path, X_test[l_sample][l_day])
            l_first_day2 += timedelta(days=1)
        l_first_day += timedelta(days=1)

    # y_test
    l_first_day = first_day + timedelta(days=3)
    for l_sample in range(y_test.shape[0]):
        sample_path = path + y_test_path + datetime.strftime(l_first_day, "%Y-%m-%d") + '/'
        Path(sample_path).mkdir(parents=True, exist_ok=True)
        array_save_image(sample_path, y_test[l_sample][0])
        l_first_day += timedelta(days=1)
        
    # pred
    l_first_day = first_day + timedelta(days=3)
    for l_sample in range(pred.shape[0]):
        sample_path = path + y_pred_path + datetime.strftime(l_first_day, "%Y-%m-%d") + '/'
        Path(sample_path).mkdir(parents=True, exist_ok=True)
        array_save_image(sample_path, pred[l_sample][0])
        
        sample_path = path + scaled_path + datetime.strftime(l_first_day, "%Y-%m-%d") + '/'
        Path(sample_path).mkdir(parents=True, exist_ok=True)
        array_save_image(sample_path, pred[l_sample][0], scaled=True)
        l_first_day += timedelta(days=1)
        
    # diff
    l_first_day = first_day + timedelta(days=3)
    for l_sample in range(diff.shape[0]):
        sample_path = path + diff_path + datetime.strftime(l_first_day, "%Y-%m-%d") + '/'
        Path(sample_path).mkdir(parents=True, exist_ok=True)
        array_save_image(sample_path, diff[l_sample])
           
        
        for l_feature in range(diff.shape[1]):
            append_list_as_row(diff_save_path + diff_save_name, [datetime.strftime(l_first_day, "%Y-%m-%d"), l_feature, mape[l_sample][l_feature], rmse[l_sample][l_feature], test_max_value[l_sample][l_feature], pred_max_value[l_sample][l_feature]])
            if (l_sample == 1 and l_feature == 44):print(test_max_value[l_sample][l_feature])
                
        l_first_day += timedelta(days=1)     
    
        
def array_save_image(path, array, scaled=False):
    for channel in range(array.shape[0]):
        if scaled:
            new_array = array[channel]
            new_array[new_array >= 0] *= 255
            new_array = np.asarray(new_array)
            save_grid(path + str(channel) + ".png", new_array, kernel=False)
        else:
            save_grid(path + str(channel) + ".png", array[channel], kernel=False)    

In [19]:
#### save results

#### 확진자

path = name + pred_path
save_prediction_image(path, X_test, y_test, pred, start_day2)

### figure

# path = name + pred_path + figure_path
# save_prediction_image(path, X_figure, y_figure, pred_figure, start_figure)


1.0
12.29039297460175
75.13805872771957
31.375824584385025
79.25488212777117
66.50430943381086
15.203278201001126
21.0
1.0
1.0
1.0
47.21435439572111
113.87666478904578
1.0
135.27738179778817
34.807343718227415
88.66954986387145
1.0
1.0
1.0
1.0
1.0
1.0
7.29585325277197
114.6441343225872
1.0
11.0
6.0
1.0
1.0
16.0
11.0
33.49277738617519
74.23356511771668
26.429381274340052
62.28960556387538
11.0
6.0
1.0
1.0
1.0
75.74226591511746
81.58301214292804
41.475155841169894
1.0
1.0
12.29039297460175
78.62187355206339
29.36097664290734
65.00203237036955
52.94346264377227
11.925498496748345
23.035364693592143
1.0
1.0
1.0
41.59229394353208
99.99657826869267
1.0
111.68943040905424
32.91558794640474
90.79357626489102
1.0
1.0
1.0
1.0
1.0
1.0
1.0
81.0
1.0
11.0
6.0
1.0
1.0
11.0
11.0
32.42902807610959
50.77909458818521
22.96100573621343
62.28960556387538
7.202561236789037
6.0
1.0
1.0
1.0
1.0
81.58301214292804
41.475155841169894
50.09811066249432
50.09811066249432
1.0
8.574347703709371
59.86308166399227
20.

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


### README

In [ ]:
# ```
read = ['- dataset info',
        '  - start_date: ',
        '  - end_date: ',
        '  - duration: ',
        '- setting data preprocessing',
        '  - image_size: ',
        '  - kde_bandwidth: ',
        '  - weight_per_person: ',
        '- setting feature',
        '  - feature_names: ',
        '  - feature_counts: ',
        '  - visit_types: ',
        '  - causes: ',
        '- setting model',
        '  - epochs: ',
        '  - batch_size: ',
        '  - optimizer: ',
        '  - loss: ', 
        '  - activation: ',
        '  - #train, #test: ',
        'extended 3 days of figure set']

read[1] += datetime.strftime(first_day, "%Y-%m-%d")
read[2] += datetime.strftime(last_day, "%Y-%m-%d")
read[3] += str(duration)

read[5] += str(size)
read[6] += str(kernel_size)
read[7] += str(weight)

counts = map(str, counts)
read[8] += ', '.join(names)
read[10] += ', '.join(counts)
read[11] += ', '.join(visit_types)
read[12] += ', '.join(causes)

sample_num = duration - (2 * n_step)
train_num = int(sample_num * 0.7)
test_num = sample_num - train_num
read[14] += str(epochs)
read[15] += str(batch_size)
read[16] += optimizer
read[17] += loss
read[18] += activation
read[18] += str(train_num) + ", " + str(test_num)

read = '\n'.join(read)

with open(name + readme_path, 'w') as f:
    f.write(read)
# ```    